In [3]:
import pandas as pd
import os
import sys
from configs import config
import json

In [3]:
matcher = AddressMatcher()
llm = load_llm(config.OPENAI_API_KEY, 0, "gpt-4-turbo-preview")

In [4]:
address = matcher.get_output_address(llm, '438, Nguyễn Tri Phương, Phường 8, Quận 10, Tp Hồ Chí Minh')
address

d:\jobs\HV gr\address_matching\src\llm_openai.py:91: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt.format(input_address=input_address))


{'amenity': '', 'street': '438 Đường Nguyễn Tri Phương', 'ad4': ' ', 'ad3': 'Phường 8', 'ad2': 'Quận 10', 'ad1': 'Thành phố Hồ Chí Minh', 'country_code': 'vn'}
{'road': 'Nguyễn Tri Phương', 'quarter': 'Phường 8', 'suburb': 'Quận 10', 'city': 'Thành phố Hồ Chí Minh', 'ISO3166-2-lvl4': 'VN-SG', 'postcode': '72415', 'country': 'Việt Nam', 'country_code': 'vn'}
num False


{'address': 'Nguyễn Tri Phương, Phường 8, Quận 10, Thành phố Hồ Chí Minh, Việt Nam',
 'structured_address': {'road': 'Nguyễn Tri Phương',
  'quarter': 'Phường 8',
  'suburb': 'Quận 10',
  'city': 'Thành phố Hồ Chí Minh',
  'ISO3166-2-lvl4': 'VN-SG',
  'postcode': '72415',
  'country': 'Việt Nam',
  'country_code': 'vn'}}

In [ ]:
def get_foreign_address_id(self, structured_address, country_code='ph'):
    if isinstance(structured_address, dict) and structured_address.get('country_code', None) == country_code:
        structured_address.pop('country_code')
        structured_address.pop('country')
        address_components = list(structured_address.values())
    else:
        print(isinstance(structured_address, dict))
        print(structured_address)
        return None
    
    address_components.reverse()
    print(address_components)

    with open(os.path.join("assets", 'AddressPhil.json'), 'r', encoding='utf_8') as file:
        prov_dicts = json.load(file)

    # ad_list = AddressJSON().get_json(country_code)
    # if prov_dicts.get('error', None):
    #     return prov_dicts

    count = 0
    ids = {}
    for address_component in address_components:
        print('aa', address_component)
        if count==0:
            city_dicts = [x for x in prov_dicts if address_component.lower() in x.get("provName", None).replace('-',' ').lower()]
            if len(city_dicts)>0:
                count+=1
                ids["provId"] = city_dicts[0].get("provId", None)
                ids["provName"] = city_dicts[0].get("provName", None)
            print(count , address_component)
            continue 
        if count==1:
            area_dicts = [x for x in city_dicts if address_component.split(' ')[-1].lower() in x.get("cityName", None).replace('-',' ').lower()]
            if len(area_dicts)>0:
                count+=1
                ids["cityId"] = area_dicts[0].get("cityId", None)
                ids["cityName"] = area_dicts[0].get("cityName", None)
            print(count , address_component)
            print(area_dicts)
            continue 
        if count==2:
            area = [x for x in area_dicts if address_component.split(' ')[-1].lower() in x.get("areaName", None).replace('-',' ').lower()]
            if len(area)==1:
                return area[0]
    if count>0:
        return ids
    return None


In [12]:
address = {
  "address": "Belisario Heights Subdivision, Doña Socorro Street, Salcedo, Tordesillas Street, Bel-Air, Metro Manila, District I, Makati, Southern Manila District, Pilipinas",
  "structured_address": {
    "building": "Barangay Bel-Air Salcedo Tanod Office",
    "road": "Tordesillas Street",
    "neighbourhood": "Salcedo",
    "quarter": "Bel-Air",
    "city_district": "District I",
    "city": "Makati",
    "state_district": "Southern Manila District",
    "region": "Metro Manila",
    "ISO3166-2-lvl3": "PH-00",
    "postcode": "1227",
    "country": "Pilipinas",
    "country_code": "ph"
  }
}

print(get_foreign_address_id('a', address['structured_address'], 'ph'))

['1227', 'PH-00', 'Metro Manila', 'Southern Manila District', 'Makati', 'District I', 'Bel-Air', 'Salcedo', 'Tordesillas Street', 'Barangay Bel-Air Salcedo Tanod Office']
aa 1227
0 1227
aa PH-00
0 PH-00
aa Metro Manila
1 Metro Manila
aa Southern Manila District
1 Southern Manila District
[]
aa Makati
2 Makati
[{'id': '45016', 'provId': '-6', 'provName': 'Metro-manila', 'cityId': '-6', 'cityName': 'Makati', 'areaId': '-6', 'areaName': 'Bangkal', 'address': 'Bangkal, Makati, Metro-manila'}, {'id': '45017', 'provId': '-6', 'provName': 'Metro-manila', 'cityId': '-6', 'cityName': 'Makati', 'areaId': '-6', 'areaName': 'Bel~air', 'address': 'Bel~air, Makati, Metro-manila'}, {'id': '45018', 'provId': '-6', 'provName': 'Metro-manila', 'cityId': '-6', 'cityName': 'Makati', 'areaId': '-6', 'areaName': 'Bel-air', 'address': 'Bel-air, Makati, Metro-manila'}, {'id': '45019', 'provId': '-6', 'provName': 'Metro-manila', 'cityId': '-6', 'cityName': 'Makati', 'areaId': '-6', 'areaName': 'Carmona', 'addr

My repo looks like this:
/address_matching
│── src/
│   ├── main.py  # FastAPI entry point
│   ├── models/
│   │   ├── __init__.py  
│   │   ├── address_json.py  
│   │   ├── address_utils.py 
│   │   ├── llm_openai.py 
│   ├── views/
│   │   ├── __init__.py  # Exposes `router`
│   │   ├── router.py  # Combines all routes
│   │   ├── address_view.py  # Address-specific API endpoints
│   ├── controllers/
│   │   ├── __init__.py
│   │   ├── address_controller.py  # Business logic for address
│── assets/
│── configs.py
│── .env

I cannot import configs in address_utils.py :
Traceback (most recent call last):
  File "d:\jobs\HV gr\address_matching\src\models\address_utils.py", line 8, in <module>
    from configs import config
ModuleNotFoundError: No module named 'configs'


In [62]:
with open(os.path.join("assets", 'AddressPhil.json'), 'r', encoding='utf_8') as file:
    ph_dict = json.load(file)


In [63]:
province = 'Metro Manila'
city_dict = [x for x in ph_dict if province.lower() in x.get("provName", None).replace('-',' ').lower()]
city_dict

[{'id': '44779',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Binondo',
  'areaId': '-6',
  'areaName': 'Barangay 282',
  'address': 'Barangay 282, Binondo, Metro-manila'},
 {'id': '44780',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Binondo',
  'areaId': '-6',
  'areaName': 'Barangay 287',
  'address': 'Barangay 287, Binondo, Metro-manila'},
 {'id': '44781',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Binondo',
  'areaId': '-6',
  'areaName': 'Barangay 288',
  'address': 'Barangay 288, Binondo, Metro-manila'},
 {'id': '44782',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Binondo',
  'areaId': '-6',
  'areaName': 'Barangay 289',
  'address': 'Barangay 289, Binondo, Metro-manila'},
 {'id': '44783',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Binondo',
  'areaId': '-6',
  'areaName': 'Barangay 290',
  'addre

In [64]:
city = 'Pasig'
area_dict = [x for x in city_dict if city.lower() in x.get("cityName", None).replace('-',' ').lower()]
area_dict

[{'id': '45533',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Pasig',
  'areaId': '-6',
  'areaName': 'Bagong ilog',
  'address': 'Bagong ilog, Pasig, Metro-manila'},
 {'id': '45534',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Pasig',
  'areaId': '-6',
  'areaName': 'Bagong katipunan',
  'address': 'Bagong katipunan, Pasig, Metro-manila'},
 {'id': '45535',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Pasig',
  'areaId': '-6',
  'areaName': 'Bambang',
  'address': 'Bambang, Pasig, Metro-manila'},
 {'id': '45536',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Pasig',
  'areaId': '-6',
  'areaName': 'Buting',
  'address': 'Buting, Pasig, Metro-manila'},
 {'id': '45537',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Pasig',
  'areaId': '-6',
  'areaName': 'Caniogan',
  'address': 'Caniogan, Pasig, Metro-manila'},

In [65]:
area = 'San Antonio'
area = [x for x in area_dict if area.lower() in x.get("areaName", None).replace('-',' ').lower()]
area

[{'id': '45551',
  'provId': '-6',
  'provName': 'Metro-manila',
  'cityId': '-6',
  'cityName': 'Pasig',
  'areaId': '-6',
  'areaName': 'San antonio',
  'address': 'San antonio, Pasig, Metro-manila'}]

In [35]:
area_list =  [x.get("areaName", None).replace('-',' ').lower() for x in area_dict]
area_list = list(set(area_list))
area_list

['calanan',
 'naneng',
 'bado dangwa',
 'ipil',
 'agbannawag',
 'malin awa',
 'san julian',
 'masablang',
 'magnao',
 'calaccad',
 'bulanao norte',
 'dilag',
 'bagumbayan',
 'lucog',
 'lanna',
 'casigayan',
 'appas',
 'dupag',
 'malalao',
 'balong',
 'bulo',
 'balawag',
 'nambaran',
 'guilayon',
 'suyang',
 'new tanglag',
 'tuga',
 'san juan',
 'laya east',
 'cudal',
 'bantay',
 'cabaritan',
 'gobgob',
 'laya west',
 'magsaysay',
 'nambucayan',
 'cabaruan',
 'dagupan weste',
 'amlao',
 'dagupan centro (pob.)',
 'bulanao',
 'lacnog']